In [ ]:
import pandas as pd
import os
import re

combined_df=pd.DataFrame()
directoryPath = r"/Users/richren/Downloads/prodpages"

def extract_urls(page_source):
            urls = re.findall(r'href="(.*?)"', page_source)
            urls = list(set([x for x in urls if ("priip" in x) or ("kiid" in x) or ("kild" in x) ]))
            return urls

for filename in os.listdir(directoryPath):
    if 'klidPageSources' in filename or 'kiidPageSources' in filename:
#         print(filename)
        file_path=os.path.join(directoryPath,filename)
        df=pd.read_pickle(file_path)
        df["source_file"]=filename
        df["kiid"]=df["pageSource"].apply(extract_urls)
        df=df.drop("pageSource",axis=1)
        combined_df=pd.concat([combined_df,df],ignore_index=True)

combined_df


In [ ]:
# combined_df.to_pickle("CombinedOut.pkl")

In [7]:
import pandas as pd 

combined_df=pd.read_pickle("CombinedOut.pkl")

In [42]:
# main loop used to scrape PDF content from every identified KIID link
from multiprocessing.pool import ThreadPool as Pool
import time
import random
import os
import math
import requests
import PyPDF2
import io
from datetime import datetime

# Function to apply
def pdfScrape(slices=0):

    import time
    from datetime import datetime
    print(slices)

    now = datetime.now()
    holder = []

    for index, row in (combined_df[slices]).iterrows():
        now = datetime.now()
        time.sleep(random.uniform(.2, 1.2))

        try:
            y = row['kiid'][0]
            # Fetch the PDF file
            response = requests.get(f"https://www.blackrock.com{y}")
            response.raise_for_status()

            # Create a PDF file object
            pdf_file = io.BytesIO(response.content)

            # Read the PDF file
            pdf_reader = PyPDF2.PdfReader(pdf_file)

            # Extract text from each page
            textList = []
            
                    
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text = page.extract_text()
                textList.append(text)
            joined = " ".join(textList)
            holder.append([y, joined, now])
        except Exception as e:
            print(e)
            holder.append([row["url"], "error", now])
            pass

    pd.DataFrame(holder).to_pickle(f'pdfScrape_{slices}.pkl')
    print(slices, "completed")

if __name__ == '__main__':
    # batch each instance into 40 url visits
    batches = 50
    # calculate number of slices using batch size above
    dfLen = combined_df.shape[0]
    rounds = math.ceil(dfLen / batches)
    # create slices
    slices = [str(slice(x * batches, (x + 1) * batches)) for x in range(rounds)]
#     print(slices)
    # lets check the directory first to make sure we havent already created these
    # from a previous run
    files = os.listdir(r"/Users/richren")

    existing = []
    for file in files:
        if "pdfScrape_" in file:
            existing.append(file.split("_")[1].split(".")[0])
    # remove those slices from the list to only do new pulls
    slices = list(set(slices) - set(existing))
    slices = [eval(x) for x in slices]
    print(slices)
#     with Pool(8) as pool:
#         df_list = pool.map(pdfScrape, slices)
#         pool.close()
#         pool.join()



[]


In [43]:

import pickle

directoryPath = r"/Users/richren"

def read_pkl_files(directory):
    combined_list = []
    for filename in os.listdir(directory):
        if 'pdfScrape_slice' in filename:
            with open(os.path.join(directory, filename), 'rb') as file:
                data = pickle.load(file)
                df = pd.DataFrame(data)
                df["file"] = "rr_"+filename
                combined_list.append(df)
    return pd.concat(combined_list)

tempdf = read_pkl_files(directoryPath)
tempdf
# tempdf=pd.read_pickle(r"/Users/richren/pdfScrape_slice(32000, 32050, None).pkl")
tempdf.columns=["KIIDurl","kiid_text","time","SourceFile"]
# tempdf["kiidISIN"]=tempdf["text"].str.extract(r'ISIN:\s*([^\s]{12})')


,KIIDurl,kiid_text,time,SourceFile
0,/dk/individual/literature/kiid/eu-priips-bgf-g...,1Key Information Document\nPurpose\nThis docum...,2024-11-23 09:30:40.004043,"rr_pdfScrape_slice(5650, 5700, None).pkl"
1,/dk/individual/literature/kiid/eu-priips-bsf-g...,1Key Information Document\nPurpose\nThis docum...,2024-11-23 09:30:41.087057,"rr_pdfScrape_slice(5650, 5700, None).pkl"
2,/dk/individual/literature/kiid/eu-priips-bgf-s...,1Key Information Document\nPurpose\nThis docum...,2024-11-23 09:30:43.557978,"rr_pdfScrape_slice(5650, 5700, None).pkl"
3,/dk/individual/literature/kiid/eu-priips-bgf-g...,1Key Information Document\nPurpose\nThis docum...,2024-11-23 09:30:45.803764,"rr_pdfScrape_slice(5650, 5700, None).pkl"
4,/dk/individual/literature/kiid/eu-priips-ishar...,1Key Information Document\nPurpose\nThis docum...,2024-11-23 09:30:48.350803,"rr_pdfScrape_slice(5650, 5700, None).pkl"
...,...,...,...,...
45,https://www.blackrock.com/de/professionelle-an...,error,2024-11-24 07:44:13.157070,"rr_pdfScrape_slice(7450, 7500, None).pkl"
46,https://www.blackrock.com/de/professionelle-an...,error,2024-11-24 07:44:13.656278,"rr_pdfScrape_slice(7450, 7500, None).pkl"
47,https://www.blackrock.com/de/professionelle-an...,error,2024-11-24 07:44:14.692619,"rr_pdfScrape_slice(7450, 7500, None).pkl"
48,https://www.blackrock.com/de/professionelle-an...,error,2024-11-24 07:44:15.688659,"rr_pdfScrape_slice(7450, 7500, None).pkl"


In [45]:
tempdf.to_pickle("scrapedPDF.pkl")

In [ ]:
combined_df["cc"]=combined_df["url"].str.split("/").str[3]
combined_df.groupby(by="cc").count()

In [ ]:
# countries=[nl,se,fi,dk,no]
import pickle

directoryPath = r"/Users/richren"

def read_pkl_files(directory):
    combined_list = []
    for filename in os.listdir(directory):
        if '_urls.pkl' in filename:
            with open(os.path.join(directory, filename), 'rb') as file:
                data = pickle.load(file)
                df = pd.DataFrame(data)
                df["file"] = filename
                combined_list.append(df)
    return combined_list

dfList = read_pkl_files(directoryPath)
dfList
linkdsDF = pd.concat(dfList)
linkdsDF.columns = ["rootSuff", "country"]
linkdsDF["countryCode"] = linkdsDF["country"].str.split("_").str[0]
linkdsDF.groupby(by="countryCode").count()
# rootUrlsDF = pd.DataFrame(rootUrls)
# rootUrlsDF.columns = ["countryCode", "root"]
# urlsWithRoots = linkdsDF.merge(rootUrlsDF, on="countryCode", how="left")
# urlsWithRoots["fullUrl"] = urlsWithRoots["root"] + urlsWithRoots["rootSuff"]
# urlsWithRoots

In [ ]:
filtered_df = combined_df[combined_df['kiid'].apply(len) == 0]
filtered_df

In [ ]:
for i in sorted(filtered_df.url.tolist()):
    print(i)

In [ ]:
filtered_df["prod"]=filtered_df.url.str.split("/").str[-1]
filtered_df.groupby("prod").count()

In [ ]:
import multiprocessing
multiprocessing.cpu_count()